## Decision Tree V1
- all datetime feature
- random cross validation

**Hyperparameter for tweeking:**
- maximale Baumtiefe
- Mindestanzahl an Daten pro Blatt
- Mindestanzahl an Daten pro Knoten
- Features

**Regularisierung:**
- Pruning

#### **_PREPARATION_**

In [1]:
# GET ALL THE JSONS INTO ONE DATAFRAME
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import json
import glob

In [2]:
# Set the search path for files (assuming the directory is relative to the current script)
file_path_mc124 = os.path.join("..", "mc124_data", "*.json")
files = glob.glob(file_path_mc124)

# Create empty list to store dataframes
li_all_files = []

# Loop through list of files and read each one into a dataframe and append to list
for f in files:
    # Read in json
    temp_df = pd.read_json(f)
    # Append df to list
    li_all_files.append(temp_df)

# Optionally concatenate all dataframes into one if needed
if li_all_files:
    combined_df = pd.concat(li_all_files)
    print(f'Combined dataframe shape: {combined_df.shape}')
else:
    print('No dataframes were created.')

Combined dataframe shape: (542555, 6)


In [3]:
combined_df.sample()

,datetime,station,core,component,period,value
1867,2021-06-15 10:00:00+02:00,mc124,no2,no2_1h,1h,48.0


In [4]:
# FILTER FOR PM10 ONLY
combined_df_pm10 = combined_df[(combined_df['core'] == 'pm10')]
combined_df_pm10.sample(4)

,datetime,station,core,component,period,value
2690,2018-05-09 13:00:00+02:00,mc124,pm10,pm10_1h,1h,45.0
3530,2017-10-02T13:00:00+02:00,mc124,pm10,pm10_1h,1h,20.0
3015,2021-05-06 20:00:00+02:00,mc124,pm10,pm10_1h,1h,17.0
2955,2022-01-07 08:00:00+01:00,mc124,pm10,pm10_1h,1h,13.0


In [5]:
# FILTER BY PARTICLE AND ONLY KEEP THE DATETIME, STATION, PERIOD AND VALUE FEATURE SINCE THE REST ARE CONSTANT INFORMATION (station, core, component, period)
df_reduced = combined_df_pm10[['datetime', 'station', 'core', 'value']]
df_reduced.sample(3)

# CUT OFF THE TIMEZONE INFORMATION FROM THE DATETIME TO AVOID CONVERSION ISSUES DUE TO TIME CHANGE IN MARCH AND OCTOBER
df_reduced.loc[:, 'datetime'] = df_reduced['datetime'].astype(str).str.slice(0, 19)
#df_pm10_reduced.loc[:, 'datetime'] = pd.to_datetime(df_pm10_reduced['datetime'], format='mixed')
df_reduced['datetime'] = pd.to_datetime(df_reduced['datetime'], format='mixed')
df_reduced.loc[:, 'datetime'] = df_reduced['datetime'].dt.tz_localize(None)
df_reduced.info()

<class 'pandas.core.frame.DataFrame'>
Index: 72314 entries, 0 to 3650
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   datetime  72314 non-null  datetime64[ns]
 1   station   72314 non-null  object        
 2   core      72314 non-null  object        
 3   value     71471 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 2.8+ MB


C:\Users\a_n_n\AppData\Local\Temp\ipykernel_6508\4237958541.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reduced['datetime'] = pd.to_datetime(df_reduced['datetime'], format='mixed')


In [6]:
df_reduced.sample(3)

,datetime,station,core,value
2255,2019-09-12 04:00:00,mc124,pm10,15.0
2195,2021-04-12 16:00:00,mc124,pm10,18.0
260,2018-04-28 19:00:00,mc124,pm10,18.0


In [7]:
#use loc to add it to every for every row
df_reduced['hour'] = df_reduced['datetime'].dt.strftime('%H')  # Hour (00-23)
df_reduced['day'] = df_reduced['datetime'].dt.strftime('%d')  # Day of the month (01-31)
df_reduced['month'] = df_reduced['datetime'].dt.strftime('%m')  # Month (01-12)
df_reduced['year'] = df_reduced['datetime'].dt.strftime('%Y')  # Month (01-12)
df_reduced.sample(3)

C:\Users\a_n_n\AppData\Local\Temp\ipykernel_6508\493841492.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reduced['hour'] = df_reduced['datetime'].dt.strftime('%H')  # Hour (00-23)


,datetime,station,core,value,hour,day,month,year
3520,2022-12-02 15:00:00,mc124,pm10,32.0,15,02,12,2022
2355,2017-02-09 08:00:00,mc124,pm10,58.0,08,09,02,2017
1464,2016-05-16 17:00:00,mc124,pm10,12.0,17,16,05,2016


In [8]:
# add day of the week to dataframe
import calendar

days = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday",
}

df_daytime = df_reduced
# convert the 'day', 'month', and 'year' columns to integers
df_daytime['day'] = df_reduced['day'].astype(int)
df_daytime['month'] = df_reduced['month'].astype(int)
df_daytime['year'] = df_reduced['year'].astype(int)

# function to determine the day of the week
def get_day_of_week(row):
    return calendar.weekday(row['year'], row['month'], row['day'])

# function to determine if day is weekday or weekend 
def is_weekend(day_number):
    return 1 if day_number >= 5 else 0
    #day_number >= 5 ? 1 : 0
    
# apply the functions to create the new columns
df_daytime['day_of_week'] = df_daytime.apply(get_day_of_week, axis=1)
df_daytime['is_weekend'] = df_daytime['day_of_week'].apply(is_weekend)

df_daytime.rename(columns={'value': 'pm10_value'}, inplace=True)

df_daytime.info()

<class 'pandas.core.frame.DataFrame'>
Index: 72314 entries, 0 to 3650
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   datetime     72314 non-null  datetime64[ns]
 1   station      72314 non-null  object        
 2   core         72314 non-null  object        
 3   pm10_value   71471 non-null  float64       
 4   hour         72314 non-null  object        
 5   day          72314 non-null  int32         
 6   month        72314 non-null  int32         
 7   year         72314 non-null  int32         
 8   day_of_week  72314 non-null  int64         
 9   is_weekend   72314 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int32(3), int64(2), object(3)
memory usage: 5.2+ MB


In [9]:
df_daytime.dropna(inplace=True)
df_daytime.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71471 entries, 0 to 3650
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   datetime     71471 non-null  datetime64[ns]
 1   station      71471 non-null  object        
 2   core         71471 non-null  object        
 3   pm10_value   71471 non-null  float64       
 4   hour         71471 non-null  object        
 5   day          71471 non-null  int32         
 6   month        71471 non-null  int32         
 7   year         71471 non-null  int32         
 8   day_of_week  71471 non-null  int64         
 9   is_weekend   71471 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int32(3), int64(2), object(3)
memory usage: 5.2+ MB


#### **_ACTUAL MODEL TRAINING_**


#### Part 1: create decision tree on everything

In [10]:
# create features and to be predicted value
y = df_daytime['pm10_value']
X = df_daytime[['hour', 'day', 'month', 'year', 'day_of_week', 'is_weekend']]

In [11]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn import tree

In [12]:
dt = DecisionTreeRegressor(max_depth=2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=1)
dt.fit(X_train, y_train)
dt.score(X_train, y_train)

0.05238106162731915

In [13]:
dt.score(X_test, y_test)

0.035010483738322384

In [22]:
dt = DecisionTreeRegressor(max_depth=30)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=1)
dt.fit(X_train, y_train)
dt.score(X_train, y_train)

0.9999993878274104

In [23]:
dt.score(X_test, y_test)
# R2 score: vergleichen Modellvorhersagen mot der Abweichung vom Mittelwert, im Vergleich mit absoluten banalsten Mittelwert vorhersage
# d.h. Vergleicht die Vorhersage des Modells vs einfach den absoluten Mittelwert für die X vorherzusagen

0.7285672945348485

In [16]:
dt.tree_.max_depth

32

#### CROSS-VALIDATION TEMPLATE

In [24]:
# cross validate
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_absolute_error, mean_squared_error, median_absolute_error, root_mean_squared_error

# Create a DecisionTreeRegressor
decision_tree = DecisionTreeRegressor(max_depth=30)

# Define your scoring functions
scorers = {
    'mean_absolute_error': make_scorer(mean_absolute_error, greater_is_better=False),
    'median_absolute_error': make_scorer(median_absolute_error, greater_is_better=False),
    'root_mean_squared_error': make_scorer(root_mean_squared_error, greater_is_better=False),
    'mean_squared_error': make_scorer(mean_squared_error, greater_is_better=False)
}
# Perform cross-validation
# scores = cross_validate(decision_tree, X, y, cv=2, scoring=scorers, return_train_score=True)
scores = cross_validate(decision_tree, X, y, cv=5, scoring=scorers, return_train_score=True)

# Print the results
print("Cross-validation scores:")
for scorer, score in scores.items():
    print(f"{scorer}: {-1*np.mean(score):.2f} (+/- {np.std(score):.2f})")

Cross-validation scores:
fit_time: -0.13 (+/- 0.00)
score_time: -0.01 (+/- 0.00)
test_mean_absolute_error: 12.25 (+/- 0.19)
train_mean_absolute_error: 0.00 (+/- 0.00)
test_median_absolute_error: 8.40 (+/- 0.49)
train_median_absolute_error: -0.00 (+/- 0.00)
test_root_mean_squared_error: 18.08 (+/- 1.18)
train_root_mean_squared_error: 0.00 (+/- 0.01)
test_mean_squared_error: 328.21 (+/- 43.53)
train_mean_squared_error: 0.00 (+/- 0.00)


- overfitting deluxe: huge discrepancy between test and training error values

In [18]:
# average pm10 value for comparison to the error rates
average_pm10 = df_daytime['pm10_value'].mean()
print(f"Average PM10 value: {average_pm10:.2f}")

Average PM10 value: 22.70


In [19]:
median_pm10= df_daytime['pm10_value'].median()
print(f"Median PM10 value: {median_pm10:.2f}")

Median PM10 value: 20.00


In [20]:
print('rerun2')

rerun2


In [21]:
# Hyperparameter () tuning mit Baum, danach Baum ensemble (random forest)